In [1]:
#Author: Felix Ackon
#Computes the emissions in a Sharpley value framework
#Created: 03-13-24
#Updated: 03-15-24 to compute the cohort values for nat gas, oil, and others
#Updated: 03-16-24 to clean and streamline code
#Updated: 03-25-24 Include code to compute the shapley values in matrix form using
                    #using wang cheng liu method
#Updated: 03-26-24 Implented the generalized method and adding comments
#Updated: 03-30-24 Do sharpley value on scenario level and clean up code
#Updated: 04-04-24 Change the emission allocation to to average emittance in the group
#Updated: 04-08-24 COmpleting the changes
#Updated: 04-12-24 Make into a function

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dill as pickle
import os
import math
from matplotlib.ticker import ScalarFormatter
from pathlib import Path
from scipy.linalg import block_diag
from itertools import compress
from itertools import chain
import sys

cwd = os. getcwd()

In [2]:
#List of helper functions
def gen_type(generator):
    """Given a string of the generator name, returns a substring containing the type of generator
    
    Parameters
    -----------
    generator: str
        string of the generator name

    Returns
    -------
    str
        a string of the generator
    """
    return  generator.split("_")[1]

#Wang cheng lui shapley
#First define the semi-tensor product
def stp(A, B):
    """Give to numpy arrays A and B where A is mxn and B is pxq then this computes the semi-tensor product
    
    Parameters
    -----------
    A: array
        a mxn matrix 
    B: array
        a pxq matrix

    Returns
    -------
    matrix
        a matrix that is the result of the semi-tensor product. The dimension is m(lcm(n,p)/n)xq(lcm(n,p)/p),
        where lcm(n,p) is the least common multiple of n and p
    """
    #First identify the shape of the matrices
    [m,n] = A.shape
    [p,q] = B.shape

    #if dimensions are the same then multiple them using standard matrix product
    #However, if dimensions are not the same then we proceed with stp formula
    if n != p:
        #Find the least common multiple
        t = math.lcm(n,p)
        #Create identitiy matrix
        ident_A = np.identity(t //n)
        ident_B = np.identity(t//p)
        #Do the matrix multiplication of the kronecker product
        A = np.kron(A, ident_A)
        B = np.kron(B, ident_B)
    return np.matmul(A,B)

#Define the recursive STP
def rec_stp(x, n):
    """Given a a 2xn matrix, where n is the number of players in the game, recursively apply STP to each column
    This was useful in understanding the examples in the paper that generated the psuedo logical function for
    the structure vector
    
    Parameters
    -----------
    A: array
        a 2xn matrix 
    n: int
        the number of columns of x and players of the coop game

    Returns
    -------
    matrix
        a matrix from repeated stp products. The matrix is (2^n)x1
    """
    #Base case: if it is just 1 vector then return the vector
    if n == 1:
        return x
    else:
        #Else recursively multiple
        return(stp(x[:,0], rec_stp(x[:,1:], n-1)))


#Define the l vectorthat holds the counting coef
def rec_coef(n):
    """Given the number of players in the game, identify the vector that holds all the counting coeficients
    in the shapley value computation
    
    Parameters
    -----------
    n: int
        the number of players in the coop game

    Returns
    -------
    matrix
        a matrix of dimention 2^n x 1. It represents eta_k in page 5
    """
    #Initialize then apply recursion
    l = np.array([1, 0])
    if n == 1:
        return(l)
    else:
        return(np.append(rec_coef(n-1) + np.ones(2 ** (n-1), dtype = int),
                         rec_coef(n-1)))

#Define the matrix T_i
def shap_helper(i,n):
    """Given the player i allocation who we want to compute and the number of players in the game, 
    identify the matrix that holds the different basis vectors needed to compute the marginal cooperative
    payoffs
    
    Parameters
    -----------
    i: int
        the player i that we want to compute the allocation for
    n: int
        the number of players in the coop game

    Returns
    -------
    matrix
        a matrix of dimention 2^n x 2^(n-1). It represents T_i or equation 6
    """
    #Computed with diagonal blocks
    if i >= 1:    
        doubly_identity = np.append(np.identity(2**(n-i), dtype = int), 
                                -1 * np.identity(2**(n-i), dtype = int), axis = 0)
        dub_id_list = [doubly_identity] * (2**(i-1))
        return(block_diag(*dub_id_list))

#Define the shapley allocation for agent i
def shap_alloc_helper(i, n):
    """Given the player i allocation who we want to compute and the number of players in the game, 
    compute the shapley allocation for that player
    
    Parameters
    -----------
    i: int
        the player i that we want to compute the allocation for
    n: int
        the number of players in the coop game

    Returns
    -------
    matrix
        a 2^n x 1 matrix that will be multiplied with the  structure vector of the characteristic function (C)
        This is the summed term in equation 5 divided by n!
    """
    #Call of the helper function
    l = rec_coef(n-1)
    T = shap_helper(i,n)
    #Since we are summing we create a zero vector as our inititialization
    value = np.zeros(2 ** n, dtype = int)
    #Sum accross all coalitions not containing i
    for j in range(2 ** (n-1)):
        value += (math.factorial(l[j]) * (math.factorial(n-1-l[j])) * T[:,j])
    return(value/ math.factorial(n))

#create psuedo logical vector
def psuedo_log(x):
    """Given a coalition with 1 at index i of the vector x as player i is in the coalition and 0 as not in
    coalition, computes the psuedo logical array
    
    Parameters
    -----------
    x: array
        the coalition represented by 1 and 0

    Returns
    -------
    matrix
        a 2 x n vector that is the psuedo logical matrix. Can be inputed to in recursive stp 
    """
    #Add a new row that is just 1 - value of index i
    return(np.matrix([x, np.ones(len(x), dtype = int) - x]))

#Create a list that gives the ordering for the charateristic function
#Turns out it is just the binary numbers in reverse order
def char_order(n):
    """Given the number of players n in the coop game, give the correct order of the payoff functions 
    in the structure vector C_v of the payoff function. Needs to be in the correct order or the matrix
    multiplication wont work. It turns out it is just binary numbers in reverse order padded with 0s.
    Also key to note the 00...0 means the perfect world scenario and 11...1 is BAU
    
    Parameters
    -----------
    n: int
        the number of players in the coop game

    Returns
    -------
    list
        a list of the order of the payoffs in the vector C_v. 
    """
    
    #The format() function simply formats the input following the Format Specification mini language. 
    #The # makes the format include the 0b prefix, and the 010 size formats the output to fit in 10 characters 
    #width, with 0 padding; 2 characters for the 0b prefix, the other 8 for the binary digits.
    #order =  [None for _ in range(n**2 -1)]
    order = [None] * (2 ** n)
    for i in range(2 **n):
        order[i] = format((2 **n) - i -1, '0' + str(n) + 'b')
    return(order)

#Create a function that converts the binary numbers to boolean list
#in opposite ditection because the 0s are the ones I idealize
def to_bool_list(x):
    list = []
    #Takes a string binary
    for i in x:
        list.append(bool(1-int(i)))
    return(list)

def flatten(xss):
    return [x for xs in xss for x in xs]

In [3]:
#Static variables
#the percentile used for computing the CVaR
alpha = 0.05
quantile = 1 - alpha
#The number of scenarios
num_scen = 500

"\nnat_gas = ['NaturalGasFiredCombustionTurbine', 'NaturalGasInternalCombustionEngine',\n         'NaturalGasFiredCombinedCycle', 'NaturalGasSteamTurbine']\ncoal = ['ConventionalSteamCoal']\nnon_emit = ['Nuclear', 'Wood/WoodWasteBiomass']\nother = ['AllOther', 'OtherGases']\n\nareas = ['Coast', 'North_Central', 'South_Central', 'South',\n         'East', 'Far_West', 'North', 'West']\n"

In [4]:
#Input files
#data: /projects/PERFORM/derand_TX_emission/texas2020/2018-08-01/emission
#Posix path object of scenario simulations with carbon emissions
folder_path = Path('2018-08-01/emission')

#Filename for the area to generator mapping
#Open up the areas file 
area_fname = "texas7k_2020_gen_to_zone.csv"

In [16]:
#Compute Shapley Allocations of All Generators
def asset_alloc(num_scen, alpha, folder_path, area_fname):
    #Open all emission files
    #iterate over directory and store filenames in a list
    #Only keep the csv files and put file names in a list
    files_list = list(filter(lambda f: f.name.endswith('.csv'), folder_path.iterdir()))

    #Create a df to hold the total emissions for each hour and scenario
    full_em_df = pd.DataFrame()
    
    
    for i in range(len(files_list)):
        #For each scenario file, open and read it into a temporary df. Then append it to the 
        #previous data frame
        temp_df = pd.read_table(files_list[i], sep = ",",
                         header = 0, index_col = False,
                          usecols = ["Date", "Hour", "Generator", "CO2 Emissions metric ton", "Dispatch"],
                               nrows = 24*3)
        #label the scenario number to keep track
        temp_df["scenario"] = files_list[i].name[5:9]
        full_em_df =  pd.concat([full_em_df, temp_df], ignore_index = True, sort=False)
    
    #Create new column with generator type
    full_em_df["Type"] = full_em_df["Generator"].apply(lambda x: gen_type(x))
    
    del temp_df

    #Open up the areas file 
    #import the data
    gen_zone_df = pd.read_table(area_fname, sep = ",",
                         header = 0, index_col = False)
    #Merge the two data sets
    full_em_df = full_em_df.merge(gen_zone_df, how = "left",
                                  left_on = "Generator", right_on = "GEN UID", copy = False)

    del gen_zone_df

    #Generate the cohort list
    #Cohort is the actual generator 
    #Use dict from keys to create an ordered set
    cohort = list(dict.fromkeys(full_em_df["Generator"]))
    #The cohort size

    n = len(cohort)
    #print(cohort)

    #Create a new vector holding the characteristic function matrix
    char_df = pd.DataFrame({"Hour": full_em_df["Hour"], "Type": full_em_df["Type"], 
                            "Generator": full_em_df["Generator"],
                            "Area": full_em_df["Area"], "Scenario": full_em_df["scenario"]})
    #Which column are we using to determine the cohorts
    category = "Generator"
    #Create the matrix of repeated C02 emissions metric ton
    emission_mat = np.tile(full_em_df["CO2 Emissions metric ton"].values,(2**n,1))
    #Create the matrix of repeated dispatch
    dispatch_mat = np.tile(full_em_df["Dispatch"].values,(2**n,1))
    #Merge these two together
    temp_mat = np.concatenate((emission_mat, dispatch_mat), axis = 0)
    
    #Check to make sure the merge happened correctly
    del emission_mat, dispatch_mat
    
    #Convert into  dataframe
    temp_df = pd.DataFrame(data = np.transpose(temp_mat), columns = [i + "e" for i in char_order(n)] + 
                           [i + "d" for i in char_order(n)], 
                           index = range(len(full_em_df["CO2 Emissions metric ton"].values)))
    
    #Merge the two data frames togetjer
    char_df = pd.concat([char_df, temp_df], axis = 1)
    del temp_df
    
    #Plug in the 0s for idealized cases
    #skip the first one which is the BAU case
    for i in char_order(n)[1:]:
        #Change the value of the new emission column and dispatch column to 0
        char_df.loc[char_df[category].isin(list(compress(cohort,to_bool_list(i)))),[i + "e", i + "d"]] = 0
    #Convert the last column back into t

    #Aggregate the data to scenario level to get the contribution values by scenario
    #Group the data frame by scenario and hour. The aggregation only requires summing the allocations
    #and the total emissions
    scen_df = char_df.drop(["Type", "Area", "Generator"], axis = 1).groupby(["Scenario", "Hour"]).sum()
    #Convert the last column back into nonzeros so we dont divide by 0
    scen_df[char_order(n)[-1] + "d"] = scen_df[char_order(n)[0] + "d"]
    
    #Compute the actual payoff value
    scen_df = scen_df.iloc[:,0:(2**n)]/ scen_df.iloc[:,(2**n):(2**(n+1))].values
    #Rename columns
    scen_df.columns = char_order(n)
    #Change all nonporucing assets from Nan to zeros
    scen_df = scen_df.fillna(0)
    
    
    #First get all the column vectors needed for shapley allowcation
    temp_mat = np.array([shap_alloc_helper(i+1,n) for i in range(n)])
    #Compute the shapley allocations
    scen_df = pd.concat([pd.DataFrame(data = np.matmul(scen_df.values, 
                                            np.transpose(temp_mat)), 
                           columns = cohort, 
                           index = scen_df.index),
                         scen_df], axis =1 )

    #Compute the 95th percentile for each hour accross all scenarious in the BAU case
    scen_df["VaR"] = scen_df[char_order(n)[0]].groupby(level = "Hour").transform("quantile", quantile)

    #Create a column that indicates if in the tail
    scen_df["inTail"] = scen_df[char_order(n)[0]] >= scen_df["VaR"]

    result_df = scen_df.loc[scen_df["inTail"], cohort + [char_order(n)[0]]]\
                .groupby(level = "Hour").sum() / (num_scen * (1-alpha))
    return(result_df)

In [17]:
temp =asset_alloc(num_scen, alpha, folder_path, area_fname)